<a href="https://colab.research.google.com/github/wilzamguerrero/blendz_colab/blob/main/BLNDZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/blendz_colab/blob/main/Custom/blender_icon.png?raw=true' height="50" alt="netdata"/></center>
import subprocess
import sys
import io
import contextlib
import os
import re
import json
from google.colab import drive
from IPython.display import clear_output, display, HTML
from IPython.utils import capture
import requests
import signal

try:
    import gradio as gr
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gradio"])
    import gradio as gr

config_file = '/tmp/config.json'

def save_config(data):
    with open(config_file, 'w') as f:
        json.dump(data, f)

def load_config():
    if os.path.exists(config_file):
        with open(config_file, 'r') as f:
            return json.load(f)
    else:
        return {}

config = load_config()
default_values = {
    'download_path': '/content/sample_data',
    'blender_url': 'https://download.blender.org/release/Blender4.2/blender-4.2.0-linux-x64.tar.xz',
    'blender_path': '/content/sample_data/blender-4.2.0-linux-x64',
    'filename': '',
    'engine_option': 'CYCLES CUDA',
    'start_frame': 1,
    'use_sequence': False,
    'end_frame': 24,
    'output_path': '/content/sample_data'
}

config = {**default_values, **config}

css_url = "https://raw.githubusercontent.com/wilzamguerrero/blendz_colab/main/Custom/blendz.css"
response = requests.get(css_url)
css_content = response.text

render_process = None

def descargar_archivo(path):
    return path

def comprimir_archivo(folder_path, archiving_method):
    output_file_path = re.search("^[\/].+\/", folder_path)
    output_file_path_raw = output_file_path.group(0)
    delsplit = re.search("\/(?:.(?!\/))+$", folder_path)
    folder_name = re.sub("^[\/]", "", delsplit.group(0))

    os.environ['inputDir'] = folder_path
    os.environ['outputPath'] = output_file_path_raw
    os.environ['folderName'] = folder_name

    if archiving_method == 'zip':
        command = f"7z a -tzip '{output_file_path_raw}/{folder_name}.zip' '{folder_path}'"
    elif archiving_method == '7z':
        command = f"7z a -t7z -m0=lzma -mx=9 -mfb=258 -md=32m -ms=on '{output_file_path_raw}/{folder_name}.7z' '{folder_path}'"

    os.system(command)

    if archiving_method == 'zip':
        return f"{output_file_path_raw}/{folder_name}.zip"
    else:
        return f"{output_file_path_raw}/{folder_name}.7z"

def conectar_drive():
    drive.mount('/content/drive')
    config['drive_connected'] = True
    save_config(config)
    return "Google Drive conectado en /content/drive"

def mostrar_auto_click_script():
    script = """
    function ClickConnect(){
        console.log("Working");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
    }
    setInterval(ClickConnect,60000)
    """
    return script

def instalar_blender(download_path, blender_url):
    config['download_path'] = download_path
    config['blender_url'] = blender_url
    save_config(config)

    os.makedirs(download_path, exist_ok=True)
    !wget {blender_url} -P {download_path}
    os.chdir(download_path)

    !tar xf blender-*.tar.xz
    os.remove(os.path.join(download_path, os.path.basename(blender_url)))

    os.environ["LD_PRELOAD"] = ""
    !apt remove libtcmalloc-minimal4
    !apt install libtcmalloc-minimal4
    os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"
    return "BLENDZ Installation Complete 100% ✓"

def fix_nvidia():

    !sudo apt-get purge nvidia* -y
    !sudo add-apt-repository ppa:graphics-drivers/ppa -y
    !sudo apt-get update
    !sudo apt-get install expect -y

    expect_script = """
    spawn sudo apt install nvidia-driver-510 nvidia-dkms-510 -y
    expect "Configuring keyboard-configuration"
    send "32\r"
    expect "Country of origin for the keyboard"
    send "1\r"
    expect eof
    """

    with open("install_nvidia.exp", "w") as script_file:
        script_file.write(expect_script)

    !expect install_nvidia.exp
    os.remove("install_nvidia.exp")

    return "NVIDIA fix completed successfully."

def renderizar_blender(blender_path, filename, engine_option, start_frame, use_sequence, end_frame, output_path):
    global render_process

    config.update({
        'blender_path': blender_path,
        'filename': filename,
        'engine_option': engine_option,
        'start_frame': start_frame,
        'use_sequence': use_sequence,
        'end_frame': end_frame,
        'output_path': output_path
    })
    save_config(config)

    if use_sequence:
        frame_option = f"-s {start_frame} -e {end_frame}"
    else:
        frame_option = f"-s {start_frame} -e {start_frame}"

    if engine_option == "CYCLES CUDA":
        command = f"{blender_path}/blender -b {filename} -noaudio -E CYCLES --debug-all -o {output_path}/#### {frame_option} -a -- --cycles-device CUDA"
    elif engine_option == "CYCLES OPTIX":
        command = f"{blender_path}/blender -b {filename} -noaudio -E CYCLES --debug-all -o {output_path}/#### {frame_option} -a -- --cycles-device OPTIX"
    elif engine_option == "EEVEE_NEXT CUDA":
        command = f"{blender_path}/blender -b {filename} -noaudio -E BLENDER_EEVEE_NEXT --debug-all -o {output_path}/#### {frame_option} -a -- --BLENDER_EEVEE_NEXT-device CUDA"
    elif engine_option == "EEVEE_NEXT OPTIX":
        command = f"{blender_path}/blender -b {filename} -noaudio -E BLENDER_EEVEE_NEXT --debug-all -o {output_path}/#### {frame_option} -a -- --BLENDER_EEVEE_NEXT-device OPTIX"

    render_process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    output_lines = []
    for line in iter(render_process.stdout.readline, ''):
        output_lines.append(line.strip())
        yield "\n".join(output_lines)

    render_process.stdout.close()
    render_process.wait()
    return "Renderizado completado"



js_func = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
refresh();
"""

with gr.Blocks(theme='ParityError/Interstellar', js=js_func, css=css_content) as iface:
#with gr.Blocks(theme='Taithrah/Minimal', js=js_func, css=css_content) as iface:

    gr.Markdown("# BLENDERZ")

    with gr.Tab("Instalar Blender"):
        gr.Markdown("## Instalar Blender")
        download_path = gr.Textbox(label="Ruta de Descarga", value=config['download_path'])
        blender_url = gr.Textbox(label="URL de Blender", value=config['blender_url'])
        install_button = gr.Button("Instalar Blender")
        install_output = gr.Textbox(label="Estado de la instalación")
        install_button.click(instalar_blender, inputs=[download_path, blender_url], outputs=install_output)

        gr.Markdown("## Fix de NVIDIA")
        fix_button = gr.Button("Aplicar Fix de NVIDIA")
        fix_output = gr.Textbox(label="Estado del Fix de NVIDIA")
        fix_button.click(fix_nvidia, outputs=fix_output)

    with gr.Tab("Renderizar Blender"):
        gr.Markdown("## Renderizar Blender")
        blender_path = gr.Textbox(label="Ruta de Blender", value=config['blender_path'])
        filename = gr.Textbox(label="Archivo .blend", value=config['filename'], placeholder="Especifica la ruta del archivo")
        engine_option = gr.Dropdown(label="Motor de Renderizado", choices=["CYCLES CUDA", "CYCLES OPTIX", "EEVEE_NEXT CUDA", "EEVEE_NEXT OPTIX"], value=config['engine_option'])
        start_frame = gr.Number(label="Frame de Inicio", value=config['start_frame'])
        use_sequence = gr.Checkbox(label="Usar Secuencia de Frames", value=config['use_sequence'])
        end_frame = gr.Number(label="Frame de Finalización", value=config['end_frame'])
        output_path = gr.Textbox(label="Ruta de Salida", value=config['output_path'])
        render_button = gr.Button("Renderizar")
        render_output = gr.Textbox(label="Estado del Renderizado")
        render_button.click(renderizar_blender, inputs=[blender_path, filename, engine_option, start_frame, use_sequence, end_frame, output_path], outputs=render_output)

    with gr.Tab("Conectar a Google Drive"):
        gr.Markdown("## Conectar a Google Drive")
        drive_button = gr.Button("Conectar a Google Drive")
        drive_output = gr.Textbox(label="Estado de la conexión")
        drive_button.click(conectar_drive, outputs=drive_output)

        gr.Markdown("## Ejecutar Auto Click en Colab")
        auto_click_script = gr.Textbox(value=mostrar_auto_click_script(), label="Código JavaScript para Auto Click", lines=10)
        gr.Markdown("Copia y pega este código en la consola de tu navegador para activar el auto click.")


    with gr.Tab("Comprimir Archivo/Carpeta"):
        gr.Markdown("## Comprimir un archivo o carpeta")
        folder_path = gr.Textbox(label="Ruta de la Carpeta o Archivo", placeholder="Especifica la ruta del archivo")
        archiving_method = gr.Dropdown(label="Método de Compresión", choices=["zip", "7z"], value="zip")
        compress_button = gr.Button("Comprimir")
        compress_output = gr.File(label="Archivo Comprimido")
        compress_button.click(comprimir_archivo, inputs=[folder_path, archiving_method], outputs=compress_output)

    with gr.Tab("Descargar Archivo"):
        gr.Markdown("## Descargar un archivo")
        download_path = gr.Textbox(label="Ruta del Archivo", placeholder="Especifica la ruta del archivo")
        download_button = gr.Button("Descargar")
        download_output = gr.File(label="Descargar Archivo")
        download_button.click(descargar_archivo, inputs=download_path, outputs=download_output)

output_buffer = io.StringIO()
clear_output()

with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
    iface.launch(debug=True, share=True)

#output = output_buffer.getvalue()
#for line in output.splitlines():
#    if "Running on public URL:" in line:
#        print(line.split("Running on public URL: ")[-1])
#        break

clear_output()
print('\033[38;2;177;81;242m' + 'BLENDERZ TERMINO ✓' + '\033[0m')



